# Practice notebook for regression analysis with dependent data in NHANES

This notebook will give you the opportunity to perform some analyses
using the regression methods for dependent data that we are focusing
on in this week of the course.

Enter the code in response to each question in the boxes labeled "enter your code here".
Then enter your responses to the questions in the boxes labeled "Type
Markdown and Latex".

This notebook is based on the NHANES case study notebook for
regression with dependent data.  Most of the code that you will need
to write below is very similar to code that appears in the case study
notebook.  You will need to edit code from that notebook in small ways
to adapt it to the prompts below.

To get started, we will use the same module imports and read the data
in the same way as we did in the case study:

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import statsmodels.api as sm
import numpy as np

url = "nhanes_2015_2016.csv"
da = pd.read_csv(url)

# Drop unused columns, drop rows with any missing values.
vars = ["BPXSY1", "BPXDI1", "RIDAGEYR", "RIAGENDR", "RIDRETH1", "DMDEDUC2", "BMXBMI",
        "SMQ020", "SDMVSTRA", "SDMVPSU"]
da = da[vars].dropna()

# This is the grouping variable
da["group"] = 10*da.SDMVSTRA + da.SDMVPSU

## Question 1: 

Build a marginal linear model using GEE for the first measurement of diastolic blood pressure (`BPXDI1`), accounting for the grouping variable defined above.  This initial model should have no covariates, and will be used to assess the ICC of this blood pressure measure.

In [3]:
model = sm.GEE.from_formula("BPXDI1 ~ 1", groups="group",
           cov_struct=sm.cov_struct.Exchangeable(), data=da)
result = model.fit()
print(result.cov_struct.summary())

The correlation between two observations in the same cluster is 0.031


In [4]:
model = sm.GEE.from_formula("BPXSY1 ~ 1", groups="group",
           cov_struct=sm.cov_struct.Exchangeable(), data=da)
result = model.fit()
print(result.cov_struct.summary())

The correlation between two observations in the same cluster is 0.030


__Q1a.__ What is the ICC for diastolic blood pressure?  What can you
  conclude by comparing it to the ICC for systolic blood pressure?

The ICC takes on values from 0 to 1, with 1 corresponding to "perfect clustering" -- the values within a cluster are identical, and 0 corresponding to "perfect independence" -- the mean value within each cluster is identical across all the clusters.

The ICC for diastolic blood pressure is very similar to that for systolic blood pressure. Hence, it can be concluded that the correlation of diastolic and systolic blood pressure between two observations in the same cluster is similar. 

## Question 2: 

Take your model from question 1, and add gender, age, and BMI to it as covariates.

In [5]:
# Create a labeled version of the gender variable
da["RIAGENDRx"] = da.RIAGENDR.replace({1: "Male", 2: "Female"})

model = sm.GEE.from_formula("BPXDI1 ~ RIAGENDRx + RIDAGEYR + BMXBMI",
           groups="group",
           cov_struct=sm.cov_struct.Exchangeable(), data=da)
result = model.fit()
print(result.cov_struct.summary())

The correlation between two observations in the same cluster is 0.030


__Q2a.__ What is the ICC for this model?  What can you conclude by comparing it to the ICC for the model that you fit in question 1?

The ICC for diastolic BP drops from 0.031 to 0.030. This means that when controlling for gender, age and BMI, the effect of clustering on diastolic BP is slightly less significant.

## Question 3: 

Split the data into separate datasets for females and for males and fit two separate marginal linear models for diastolic blood pressure, one only for females, and one only for males.

In [15]:
# Split the data into separate datasets for females and for males
df = da.loc[da.RIAGENDRx == "Female",:]
dm = da.loc[da.RIAGENDRx == "Male",:]

# Fit a marginal linear model for diastolic BP for females
model_female = sm.GEE.from_formula("BPXDI1 ~ RIAGENDRx + RIDAGEYR + BMXBMI",
           groups="group",
           cov_struct=sm.cov_struct.Exchangeable(), data=df)
result_female = model_female.fit()

# Fit a marginal linear model for diastolic BP for males
model_male = sm.GEE.from_formula("BPXDI1 ~ RIAGENDRx + RIDAGEYR + BMXBMI",
           groups="group",
           cov_struct=sm.cov_struct.Exchangeable(), data=dm)
result_male = model_male.fit()

x = pd.DataFrame({"female_params": result_female.params, "female_SE": result_female.bse,
                  "male_params": result_male.params, "male_SE": result_male.bse})
x = x[["female_params", "female_SE", "male_params", "male_SE"]]
print(x)

           female_params  female_SE  male_params   male_SE
Intercept      65.750510   1.347957    66.894894  1.750734
RIDAGEYR       -0.019222   0.019262    -0.066136  0.016754
BMXBMI          0.132351   0.030256     0.265171  0.054721


__Q3a.__ What do you learn by comparing these two fitted models?

By comparing these two fitted models, it can be concluded that across all clusters:
1) An increase in age results in a decrease in diastolic blood pressure for both females and males (this decrease is more significant for males)
2) An increase in BMI results in an increase in diastolic blood pressure for both females and males (this increase is more significant for males) 
3) The standard error for males tends to be higher for the parameters possibly due to the lower sample size for males or greater clustering effect seen in males. 

## Question 4: 

Using the entire data set, fit a multilevel model for diastolic blood pressure, predicted by age, gender, BMI, and educational attainment.  Include a random intercept for groups.

In [23]:
# Fit a multilevel (mixed effects) model to handle dependent data
model = sm.MixedLM.from_formula("BPXDI1 ~ RIDAGEYR + RIAGENDRx + BMXBMI + C(DMDEDUC2)",
           groups="group", data=da)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
            Mixed Linear Model Regression Results
==============================================================
Model:               MixedLM  Dependent Variable:  BPXDI1     
No. Observations:    5102     Method:              REML       
No. Groups:          30       Scale:               154.2055   
Min. group size:     106      Log-Likelihood:      -20115.7588
Max. group size:     226      Converged:           Yes        
Mean group size:     170.1                                    
--------------------------------------------------------------
                   Coef.  Std.Err.   z    P>|z|  [0.025 0.975]
--------------------------------------------------------------
Intercept          64.688    1.153 56.123 0.000  62.429 66.947
RIAGENDRx[T.Male]   2.756    0.351  7.860 0.000   2.069  3.444
C(DMDEDUC2)[T.2.0]  0.736    0.726  1.013 0.311  -0.688  2.159
C(DMDEDUC2)[T.3.0] -0.139    0.644 -0.216 0.829  -1.401  1.122
C(DMDEDUC2)[T.4.0]  0.527    0.618  0.852 0.394  -0.685  1.739
C(DMDEDUC2)[T.5.0]  0.955    0.641  1.489 0.137  -0.302  2.212
C(DMDEDUC2)[T.9.0] -2.978    8.823 -0.338 0.736 -20.272 14.315
RIDAGEYR           -0.039    0.010 -3.870 0.000  -0.059 -0.019
BMXBMI              0.186    0.026  7.286 0.000   0.136  0.236
group Var           4.171    0.108                            
==============================================================

"""

In [24]:
np.sqrt(2*4.171)

2.888252066562058

__Q4a.__ How would you describe the strength of the clustering in this analysis?

The variance for groups is estimated to be 4.171. This means that if we were to choose two groups at random, their diastolic blood pressure would differ on average by around 2.89 units (this is calculated as the square root of 2*4.171) due to random effect. This is a sizable shift, comparable to the difference between females and males, or to around 74 years of aging.

__Q4b:__ Include a random intercept for age, and describe your findings.

In [26]:
da["age_cen"] = da.groupby("group").RIDAGEYR.transform(lambda x: x - x.mean())

model = sm.MixedLM.from_formula("BPXDI1 ~ age_cen + RIAGENDRx + BMXBMI + C(DMDEDUC2)",
           groups="group", vc_formula={"age_cen": "0+age_cen"}, data=da)
result = model.fit()
result.summary()

/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


<class 'statsmodels.iolib.summary2.Summary'>
"""
            Mixed Linear Model Regression Results
==============================================================
Model:               MixedLM  Dependent Variable:  BPXDI1     
No. Observations:    5102     Method:              REML       
No. Groups:          30       Scale:               157.7332   
Min. group size:     106      Log-Likelihood:      -20159.6320
Max. group size:     226      Converged:           Yes        
Mean group size:     170.1                                    
--------------------------------------------------------------
                   Coef.  Std.Err.   z    P>|z|  [0.025 0.975]
--------------------------------------------------------------
Intercept          62.754    0.941 66.706 0.000  60.910 64.598
RIAGENDRx[T.Male]   2.821    0.354  7.959 0.000   2.126  3.515
C(DMDEDUC2)[T.2.0]  0.812    0.729  1.113 0.266  -0.618  2.241
C(DMDEDUC2)[T.3.0] -0.141    0.642 -0.220 0.826  -1.399  1.116
C(DMDEDUC2)[T.4.0]  0.707    0.614  1.151 0.250  -0.497  1.912
C(DMDEDUC2)[T.5.0]  1.339    0.630  2.125 0.034   0.104  2.575
C(DMDEDUC2)[T.9.0] -4.389    8.924 -0.492 0.623 -21.879 13.102
age_cen            -0.033    0.015 -2.185 0.029  -0.063 -0.003
BMXBMI              0.175    0.025  6.869 0.000   0.125  0.225
age_cen Var         0.004    0.000                            
==============================================================

"""

In [28]:
-0.033 - 0.06

-0.093

We see that the estimated variance for random age slopes is 0.004, which translates to a standard deviation of 0.06. That is, from one cluster to another, the age slopes fluctuate by  ±0.06−0.12  (1-2 standard deviations). These cluster-specific fluctuations are added/subtracted from the fixed effect for age, which is -0.033. Thus, in some clusters DBP may increase by around -0.033 + 0.06 = 0.027 mm/Hg per year, while in other clusters DBP may decrease by around -0.033 - 0.06 = -0.093 mm/Hg per year. Note also that the fitting algorithm produces a warning that the estimated variance parameter is close to the boundary. In this case, however, the algorithm seems to have converged to a point just short of the boundary.